In [2]:
from gensim.models import KeyedVectors

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
w2v=KeyedVectors.load("/content/drive/MyDrive/FYP/CAA_NRC/FastText/ft")

In [6]:
import pandas as pd
import numpy as np
df=pd.read_csv("/content/drive/MyDrive/FYP/CAA_NRC/CAA_NRC-preprocessed.csv")
y=df["label"]


In [ ]:
maxi=-1
for i,rev in enumerate(df['new_text']):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)
print(maxi)

In [7]:
import tensorflow as tf
maxlen=66
max_words=len(w2v.wv.vocab.keys())
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
def padded_sequence(X):
  tokenizer.fit_on_texts(X)
  sequences = tokenizer.texts_to_sequences(X)
  data = pad_sequences(sequences, maxlen=maxlen)
  return data
X=padded_sequence(df["new_text"])
X

array([[   0,    0,    0, ...,    7,   55,   69],
       [   0,    0,    0, ...,   77,  139,  871],
       [   0,    0,    0, ...,   76,   81, 1784],
       ...,
       [   0,    0,    0, ...,   48,   18,    3],
       [   0,    0,    0, ...,  223,   80,   90],
       [   0,    0,    0, ...,  315,  754,   50]], dtype=int32)

In [8]:
import numpy as np
embed_matrix=np.zeros(shape=(max_words,300))
d={}
for word,i in tokenizer.word_index.items():
  d[i]=word

In [9]:
for i in X:
  for j in range(len(i)):
    if i[j]==0:
      pass
    else:
      p=d[i[j]]
      if p in w2v:
        i[j]=sum(w2v[p])
      else:
        i[j]=0
  print(i)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Streaming output truncated to the last 5000 lines.
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0 -4 -1  0 -2 -1  0  0 -3 -1 -1 -2  0 -2  2  1 -3 -1  0  0  0  0
  0  0  1  1  0 -2 -1 -2 -1 -1  0 -1 -1 -1  0 -1 -3 -1]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1 -1 -2  0  1 -1  0 -1  0 -1
  0 -1  0 -2 -1  0 -1 -1  0  0 -3  0 -2  0 -1  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0 -2  0 -2 -4  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1  0 -1 -1  0 -1
 -1  0 -1 -1 -1  0  0  0 -1  0  0 -1 -3 -1  0  0 -2 -1]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1 -1 -3  0  

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [11]:
#Metric calculation function
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def display_metrics(y_test,y_pred):
    print(accuracy_score(y_test, y_pred))
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))

In [12]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=10000).fit(X_train, y_train)
y_pred=clf.predict(X_test)

In [13]:
display_metrics(y_test,y_pred)

0.43170103092783507
0.32543188757459074
0.39890283192549275
0.35068751008940224


In [14]:
from sklearn import svm 
clf = svm.SVC(kernel='linear',verbose=1,max_iter=10000)
clf.fit(X_train, y_train)

[LibSVM]

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


SVC(kernel='linear', max_iter=10000, verbose=1)

In [15]:
y_pred=clf.predict(X_test)
display_metrics(y_test,y_pred)

0.42396907216494845
0.29841362341362343
0.39143586250966944
0.3356074643489791


In [16]:
from sklearn import tree
DTclf = tree.DecisionTreeClassifier()
DTclf.fit(X_train, y_train)

DecisionTreeClassifier()

In [17]:
y_pred=DTclf.predict(X_test)
display_metrics(y_test,y_pred)

0.47680412371134023
0.4543339304936787
0.4566768726468206
0.45262273167292144
